# Exploración de variables espaciales
<b>Proyecto</b>:	PDB IV - Pobreza urbana
<b>Objetivo</b>:    Desarrollo de variables espaciales y tabulaciones de pobreza con variables 

### 1. Bases de datos y librerías

In [17]:
import pandas as pd
import os
import numpy as np
import geopandas as gdp
import matplotlib.pyplot as plt

In [163]:
os.chdir(r'C:\Users\User\OneDrive - MIGRACIÓN VIDENZA\1. Proyectos\1. Proyectos actuales\23. Artículos PDB\2. PDB - Pobreza Urbana\2. Data\1. Bases\2. ENAHO Anual')

mod5 = pd.read_stata('modulo500.dta', convert_categoricals=False,
                     columns= ['anio', 'mes', 'conglome', 'vivienda','hogar', 'ubigeo', 'dominio', 'estrato',
                               'codperso', 'fac500a', 'p558d2_1', 'p558d2_2', 'ocupinf', 'i524a1', 'd529t',
                               'i530a', 'd536', 'i538a1', 'd540t', 'i541a', 'd543', 'd544t', 'p521a', 'p521',
                               'p519','i520', 'i513t', 'i518'])

sumaria = pd.read_stata('sumaria.dta', convert_categoricals=False,
                        columns= ['anio', 'mes', 'conglome', 'vivienda', 'hogar', 'percepho', 'mieperho', 'gashog2d',
                                  'linpe', 'linea', 'gashog1d', 'pobrezav', 'pobreza', 'ipcr_0', 'gpgru0'])

mod1 = pd.read_stata('modulo100.dta', convert_categoricals=False,
                     columns= ['anio', 'mes', 'conglome', 'vivienda', 'hogar', 'latitud', 'longitud'])

mod2 = pd.read_stata('modulo200.dta', convert_categoricals=False,
                     colums=['anio', 'mes', 'conglome', 'vivienda', 'hogar', 'codperso', 'p208a'])

dfEnaho = pd.merge(mod5, sumaria, how='inner', on=['anio', 'mes', 'conglome', 'vivienda', 'hogar'], validate="m:1")

dfEnaho = pd.merge(dfEnaho, mod1, how='inner', on=['anio', 'mes', 'conglome', 'vivienda', 'hogar'], validate="m:1")


C:\Users\User\AppData\Local\Temp\ipykernel_15332\2898788602.py:3: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  mod5 = pd.read_stata('modulo500.dta', convert_categoricals=False,
C:\Users\User\AppData\Local\Temp\ipykernel_15332\2898788602.py:9: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  sumaria = pd.read_stata('sumaria.dta', convert_categoricals=False,
C:\Users\User\AppData\Local\Temp\ipykernel_15332\2898788602.py:13: UnicodeWarning: 
One or more strings in the dta file could not be decoded using ut

TypeError: read_stata() got an unexpected keyword argument 'colums'

In [83]:
# Nos quedamos solo con las observaciones urbanas
dfUrb = dfEnaho[dfEnaho['estrato'] < 6]

## 2. Exploración distancia al trabajo

In [84]:
# Modificamos la variable p558d2_2 para ponerlo en formato ubigeo
dfUrb.loc[:,'p558d2_2'] = dfUrb['p558d2_2'].fillna('0')
dfUrb.loc[:,'p558d2_2'] = dfUrb['p558d2_2'].astype(int).astype(str).str.zfill(6)

C:\Users\User\AppData\Local\Temp\ipykernel_15332\830759936.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['0' '0' '0' ... 150811.0 150811.0 '0']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  dfUrb.loc[:,'p558d2_2'] = dfUrb['p558d2_2'].fillna('0')


#### 2.1. Residencia en distritos atractores de empleo

In [85]:
# Generamos la variable de pctResidencia
distResidencia = dfUrb.loc[:,['ubigeo','fac500a']].dropna()
distResidencia['provResidencia'] = (distResidencia['ubigeo'].astype(int) / 100).astype(int).astype(str).str.zfill(4)
distResidencia = distResidencia.dropna()

# Contabilizamos la cantidad de personas que residen por distrito
distResidencia = distResidencia.groupby(['provResidencia',"ubigeo"])['fac500a'].sum().reset_index()

distResidencia = distResidencia.rename(columns={'fac500a':'nPersonas'})
distResidencia['nPersonasProv'] = distResidencia.groupby('provResidencia')['nPersonas'].transform('sum')
distResidencia['pctResidencia'] = distResidencia['nPersonas'] / distResidencia['nPersonasProv']

In [86]:
# Generamos la variable ubigeo del centro de trabajo
dfUrb.loc[:,'ubigeoTrabajo'] = dfUrb.apply(lambda row: row['ubigeo'] if row['p558d2_1'] == 1 else row['p558d2_2'], axis=1)
dfUrb.loc[:,'ubigeoTrabajo'] = dfUrb['ubigeoTrabajo'].replace("000000", pd.NA).replace("999999", pd.NA)

C:\Users\User\AppData\Local\Temp\ipykernel_15332\3557774175.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfUrb.loc[:,'ubigeoTrabajo'] = dfUrb.apply(lambda row: row['ubigeo'] if row['p558d2_1'] == 1 else row['p558d2_2'], axis=1)


In [87]:
# Generamos la variable provincia del trabajo 
distTrabajo = dfUrb.loc[:,['ubigeoTrabajo','ubigeo','fac500a']].dropna()
distTrabajo['provTrabajo'] = (distTrabajo['ubigeoTrabajo'].astype(int) / 100).astype(int).astype(str).str.zfill(4)
distTrabajo = distTrabajo.dropna()

# Contabilizamos la cantidad de empleos que se generan
distTrabajo = distTrabajo.groupby(['provTrabajo',"ubigeoTrabajo"])['fac500a'].sum().reset_index()

In [88]:
distTrabajo = distTrabajo.rename(columns={'fac500a':'nTrabajos'})
distTrabajo['nTrabajosProv'] = distTrabajo.groupby('provTrabajo')['nTrabajos'].transform('sum')
distTrabajo['pctTrabajo'] = distTrabajo['nTrabajos'] / distTrabajo['nTrabajosProv']

In [89]:
# Definimos como centralidad si un distrito concentra atrae más trabajo que residentes.
distTrabajo = distTrabajo.rename(columns={'ubigeoTrabajo':'ubigeo'})
distUrb = pd.merge(distTrabajo, distResidencia, how='inner', on='ubigeo')
distUrb = distUrb.loc[:, ['ubigeo', 'pctResidencia', 'pctTrabajo']]

cond1 = (distUrb['pctTrabajo'] > distUrb['pctResidencia'] + 0.01 )
distUrb['distCentral'] = 0
distUrb.loc[ cond1 ,'distCentral'] = 1

del distTrabajo, distResidencia, cond1

#### 2.2. Distancia ecluidiana al trabajo

In [90]:
dfUrb = pd.merge(dfUrb, distUrb, how='left', on='ubigeo', validate="m:1")

cond2 = (dfUrb['pobreza'] < 3 )
dfUrb['pobre'] = 0
dfUrb.loc[ cond2 ,'pobre'] = 1
del cond2

In [91]:
tabla1 = pd.crosstab([dfUrb["anio"], dfUrb["distCentral"]], dfUrb["pobre"], values=dfUrb["fac500a"], aggfunc="sum", normalize='index')
tabla1.reset_index(inplace=True)
tabla1 = tabla1.drop(0, axis=1)

In [92]:
from geopy.distance import geodesic

def distancia_euclidiana(lat1, lon1, lat2, lon2):
    coord1 = (lat1, lon1)
    coord2 = (lat2, lon2)
    return geodesic(coord1, coord2).kilometers

In [94]:
path_dist = "C:/Users/User/OneDrive - Universidad del Pacífico/1. Documentos/0. Bases de datos/11. Mapas Perú/Límite Distrital"

peru_dist = gdp.read_file(path_dist + "/LIMITE_DISTRITAL_2020_INEI_geogpsperu_juansuyo_931381206.shp")

var1 = peru_dist['UBIGEO']
var2 = peru_dist.centroid
var3 = var2.x
var4 = var2.y

centroids = pd.DataFrame({'p558d2_2': var1,
                          'x_long' : var3,
                          'y_lat' : var4})
del var1, var2, var3, var4

C:\Users\User\AppData\Local\Temp\ipykernel_15332\2172197495.py:6: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  var2 = peru_dist.centroid


In [ ]:
dfUrb = pd.merge(dfUrb, centroids, how='left', on='p558d2_2', validate="m:1")

In [109]:
dfUrb2 = dfUrb[dfUrb['p558d2_2'] != '000000']
dfUrb2 = dfUrb2[dfUrb2['y_lat'].notna()]
dfUrb2 = dfUrb2.rename(columns={'y_lat':'lat_trabajo', 'x_long':'long_trabajo'})

In [110]:
dfUrb2["distanciaTrabajo"] = pd.Series(dtype=float)

for index,row in dfUrb2.iterrows():
    lat1 = float(dfUrb2.loc[index,'latitud'])
    lng1 = float(dfUrb2.loc[index,'longitud'])
    lat2 = float(dfUrb2.loc[index,'lat_trabajo'])
    lng2 = float(dfUrb2.loc[index,'long_trabajo'])

    try:
        dfUrb2.loc[index, f'distanciaTrabajo'] = distancia_euclidiana(lat1, lng1, lat2, lng2)
    except:
        continue

## 3. Exploración distancia a escuelas públicas

In [145]:
os.chdir(r'C:\Users\User\OneDrive - MIGRACIÓN VIDENZA\1. Proyectos\1. Proyectos actuales\23. Artículos PDB\2. PDB - Pobreza Urbana\2. Data\1. Bases')

dfIIEE = pd.read_csv('listado_iiee.csv')
dfIIEE.loc[:,'Ubigeo'] = dfIIEE['Ubigeo'].astype(str).str.zfill(6)
dfIIEE.loc[:,'tipoGestion'] = dfIIEE['Gestion / Dependencia'].str[:7]


C:\Users\User\AppData\Local\Temp\ipykernel_15332\815276184.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['010101' '010101' '010101' ... '250401' '250401' '250401']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  dfIIEE.loc[:,'Ubigeo'] = dfIIEE['Ubigeo'].astype(str).str.zfill(6)


In [164]:
os.chdir(r'C:\Users\User\OneDrive - MIGRACIÓN VIDENZA\1. Proyectos\1. Proyectos actuales\23. Artículos PDB\2. PDB - Pobreza Urbana\2. Data\1. Bases\2. ENAHO Anual')
mod3 = pd.read_stata('modulo300.dta', convert_categoricals=False,
                     columns=['anio', 'mes', 'conglome', 'vivienda', 'hogar', 'codperso', 'ubigeo', 'estrato', 'p301a', 'p307', 'p308a', 'p308c1', 'p308c2', 'p208a'])

mod3 = pd.merge(mod3, mod1, how='inner', on=['anio', 'mes', 'conglome', 'vivienda', 'hogar'], validate="m:1")

C:\Users\User\AppData\Local\Temp\ipykernel_15332\3265230413.py:2: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  mod3 = pd.read_stata('modulo300.dta', convert_categoricals=False,


In [165]:
cond1 = mod3['p308a'] == 1
cond2 = mod3['p308c1'].notna()
cond3 = mod3['estrato'] < 6
mod3 = mod3[cond1 & cond2 & cond3]

In [166]:
distUrb2 = pd.Series(mod3['ubigeo'].unique(), name='Ubigeo')
dfIEUrb = pd.merge(distUrb2, dfIIEE, on='Ubigeo')
dfIEUrb = dfIEUrb[dfIEUrb['tipoGestion'] == 'Pública']
del distUrb2

In [167]:
# Modificamos la variable p308c2 para ponerlo en formato ubigeo
mod3.loc[:,'p308c2'] = mod3['p308c2'].fillna('0')
mod3.loc[:,'p308c2'] = mod3['p308c2'].astype(int).astype(str).str.zfill(6)
mod3 = mod3.rename(columns={'p308c2':'Ubigeo'})

C:\Users\User\AppData\Local\Temp\ipykernel_15332\3338076744.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['050101' '060108' '070104' ... '230109' '110106' '150811']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  mod3.loc[:,'p308c2'] = mod3['p308c2'].astype(int).astype(str).str.zfill(6)


In [168]:
dfIEUrb = pd.merge(mod3, dfIEUrb, on='Ubigeo', how='inner' , validate='m:m')

In [170]:
dfIEUrb = dfIEUrb.rename(columns={'latitud':'latCong',
                        'longitud' : 'longCong',
                        'Latitud': 'latIE',
                        'Longitud': 'longIE'})

dfIEUrb["distanciaIE"] = pd.Series(dtype=float)

for index,row in dfIEUrb.iterrows():
    lat1 = float(dfIEUrb.loc[index,'latCong'])
    lng1 = float(dfIEUrb.loc[index,'longCong'])
    lat2 = float(dfIEUrb.loc[index,'latIE'])
    lng2 = float(dfIEUrb.loc[index,'longIE'])

    try:
        dfIEUrb.loc[index, f'distanciaIE'] = distancia_euclidiana(lat1, lng1, lat2, lng2)
    except:
        continue

In [ ]:
dfIEUrb['ID'] = dfIEUrb['anio'].astype(str) + '_' + dfIEUrb['mes'].astype(str) + '_' + dfIEUrb['conglome'].astype(str) + '_' + dfIEUrb['vivienda'].astype(str) + '_' + dfIEUrb['hogar'].astype(str) + '_' + dfIEUrb['codperso'].astype(str)

listID = dfIEUrb['ID'].unique().tolist()

dfIEUrb2 = pd.DataFrame()

for id in listID:
    alterna = dfIEUrb[dfIEUrb['ID'] == id]
    alterna = alterna.sort_values(by=['distanciaIE'])
    alterna = alterna[:1]

    dfIEUrb2 = pd.concat([dfIEUrb2, alterna], ignore_index=True)